<a href="https://colab.research.google.com/github/ajkerrigan/my-itchy-dog/blob/main/Ginger_Scratching_Trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graphing Ginger's Scratching

![Ginger, medium-sized brown/white terrier mix, curled up in a comfy ball](https://github.com/ajkerrigan/my-itchy-dog/blob/4baa6f087607b2256a98c31d752b492b3595b069/images/ginger.png?raw=true "Ginger, medium-sized brown/white terrier mix, curled up in a comfy ball")

That's Ginger - she has sensitive skin and is allergic to everything, aww...

We rely on allergy medication to help manage her itchiness, and use a [Whistle](https://www.whistle.com/) device to help track her scratching frequency over time. That's a very useful piece of the puzzle, but it can only show 1 week or 1 month of scratching data at a time. To gauge medication effectiveness and help plan future doses, it's helpful to look further back to see longer-term trends.

Here we fetch data from the Whistle API (cheers to [go-whistle-wrapper](https://github.com/amattu2/go-whistle-wrapper) for highlighting the methods I needed), then wrap it in a [Polars](https://pola-rs.github.io/) data frame and graph it with [Altair](https://altair-viz.github.io).

Thanks to [Russell Helmstedter](https://github.com/rhelmstedter)'s Python class for insightful questions and feedback about this notebook. Some led directly to changes, like:

- Outputting a sample of raw data before going right to a graph
- Parameterizing values like pet name and how far back to look in history
  - ...and realizing just how ludicrously far back in time Whistle will let you look 😳

So thanks to them! 👋 ♥

# Step 0 - Dependencies


This uses Altair's [experimental support](https://altair-viz.github.io/user_guide/data.html#specifying-data) for the DataFrame Interchange Protocol under the hood, so step 0 is making sure we've got current versions of altair/pyarrow/polars installed.

In [1]:
%pip install --upgrade altair pyarrow polars

# Fetch the Data

Grab 180 days of scratching data from the Whistle API, and wrap it in a Polars DataFrame.

To adapt this notebook for use with a new Whistle account and pet:

- Update the Google Colab notebook secrets
  - `email` is the email address associated with an active Whistle account
  - `refresh_token` comes from a prior login request. If you don't have one, adjust the code to use `password` instead.
- Adjust the `pet_name` and `days_of_history` parameters as needed

In [2]:
import polars as pl
import requests

from google.colab import userdata

# Hi it's me
auth_token = requests.post(
    'https://app.whistle.com/api/login',
    json={
        'email': userdata.get('email'),
        'refresh_token': userdata.get('refresh_token')
    }
).json()['auth_token']

# Where my dogs at?
pets = requests.get(
    'https://app.whistle.com/api/pets',
    headers={
        'Accept': "application/vnd.whistle.com.v4+json",
        'Authorization': f'Bearer {auth_token}'
    }
).json()['pets']

# Hey that's my dog
pet_name = 'Ginger'  # @param {type:"string"}
pet_id = next(p['id'] for p in pets if p['name'] == pet_name)

# Itchiness diary please
days_of_history = 180  # @param {type:"slider", min:1, max:365, step:1}
scratching = requests.get(
    f'https://app.whistle.com/api/pets/{pet_id}/health/graphs/scratching?num_of_days={days_of_history}',
    headers={
        'Accept': "application/vnd.whistle.com.v4+json",
        'Authorization': f'Bearer {auth_token}'
    }
).json()['data']

df = pl.DataFrame(scratching)

df

start_date,score,unit,status,fraction_valid
str,i64,str,str,f64
"""2024-01-31""",35,"""second""","""infrequent""",1.0
"""2024-01-30""",63,"""second""","""occasional""",1.0
"""2024-01-29""",87,"""second""","""occasional""",1.0
"""2024-01-28""",36,"""second""","""infrequent""",1.0
"""2024-01-27""",60,"""second""","""occasional""",1.0
"""2024-01-26""",58,"""second""","""occasional""",1.0
"""2024-01-25""",73,"""second""","""occasional""",1.0
"""2024-01-24""",62,"""second""","""occasional""",1.0
"""2024-01-23""",115,"""second""","""occasional""",1.0


# Graph

Whistle has four categories of scratching, color coded as green/yellow/orange/red in the app. That's a useful breakdown, so use the same scheme here.

Aside from the custom color scale and some friendly titles, this is right out of the Altair [getting started](https://altair-viz.github.io/getting_started/overview.html) material.

In [3]:
import altair as alt

scale = alt.Scale(
    domain=['infrequent', 'occasional', 'elevated', 'severe'],
    range=['green', 'yellow', 'orange', 'red']
)

alt.Chart(df, title="Ginger's Scratching").mark_bar().encode(
    x=alt.X('start_date', type='temporal').title('Date'),
    y=alt.Y('score', type='quantitative').title('Scratching Time (seconds)'),
    color=alt.Color('status', type='nominal', scale=scale)
).properties(
    width='container',
    height=600,
)

alt.Chart(...)